In [1]:
# Install our (local, for now) pip package 
!pip3 install -e /home/jjhenkel/find-tune

In [1]:
import mlos

OVERRIDE_PLUGIN='/home/jjhenkel/mount/velveeta/find-tune/frontends/clang/src/libDumpDatalog.so'

mlos.set_project_dir('/home/jjhenkel/leveldb/.mlos')
mlos.checkenv(using_azure=True)

project = (
    mlos
    .create_project_from_git_info()
    .with_backend(mlos.backends().azure())
    .with_build_options(
        mlos.build_options().override_plugin(OVERRIDE_PLUGIN)
    )
)

project.attach()

- Verifying environment...
   ✓ Environment verification complete
- Creating project from current git repository...
   - Getting project name: 
      ✓ project-name:='https://github.com/jjhenkel/leveldb'
   - Getting project commit hash: 
      ✓ project-commit:='23238a40ab67edb0dc10314c28d37b8a604d14f1'
- Ensuring infrastructure is allocated...
   ✓ Found resource group 'FindTune-EUS'
   ✓ Using existing vnet: 'findtune-az-autogen-vnet-801663' (['10.0.0.0/16'])
   ✓ Using existing subnet: 'findtune-az-autogen-subnet-806943' (10.0.0.0/24)
   ✓ Using existing ip: 'findtune-az-autogen-ip-ft-db-211043' (52.226.50.179)
   ✓ Using existing vm: 'findtune-az-autogen-vm-ft-db-500381'
- Attaching...
   ✓ 'findtune-az-autogen-vm-ft-db-500381' is running!
   ✓ Skipping DB create. Database 'targets-leveldb' already exists!
   ✓ Attached. Ready to run queries


In [2]:
# Remotely builds + ingests this code for analysis
project.ingest()

- Preparing target docker image...
   ✓ Overriding plugin with '/home/jjhenkel/mount/velveeta/find-tune/frontends/clang/src/libDumpDatalog.so'
   - Building image findtune/mlos-target:1454924aacba2ea3cdf183bbed512566a600b05a...
      ✓ Image build complete!
      ✓ Push complete!
   ✓ Override cleanup complete!
- Running remote build...
   ✓ Build started! Use poll_status() to watch build progress.


In [3]:
project.poll_status()

✓ Build container finished --- waiting for completed sessions backlog to clear. 
✓ Completed sessions backlog cleared --- waiting for embeddings to be computed. 
✓ Ingest complete! Use project.detach(keep_db=True) and/or project.deallocate(keep_db=True) to cleanup build & gpu VMs. 


In [4]:
project.detach(keep_db=True)

- Detaching...
   ✓ 'findtune-az-autogen-vm-ft-embed-649821' is stopped!
   ✓ 'findtune-az-autogen-vm-ft-build-172657' is stopped!
   ✓ Detached. DB VM still attached. Call project.detach(keep_db=False) to detach.


In [2]:
test = (project
    .query()
    .calls()
    .name_semantically_similar_to('lock')
    .select_text_repr()
    .select_line_info()
    .from_files_matching("db/.*")
    .structurally_unique()
    .limit(30)
    .execute()
    .display()
)

In [5]:
test = (project
    .query()
    .calls()
    .name_contains('lock')
    .select_text_repr()
    .select_line_info()
    .from_files_matching("db/.*")
    .structurally_unique()
    .limit(30)
    .execute()
    .display()
)

In [11]:
# The targets we'd like to match against
destinations = (project
    .query()
    .integers()
    .structurally_unique()
    .make_visualizable()
    .select()
    .execute()
)

# We only care about an integer literal if it winds up
# as a parameter to one of these calls, so these are our "sources"
sources = (project
    .query()
    .calls()
    .name_semantically_similar_to('sleep', threshold=-20.0)
    .with_single_parameter()
    .parameters()
    .select()
    .execute()
)

# Shows us integer literals that were ended up as parameters
# to one of the calls like sleep/wait/delay
(project
    .flows()
    .src(sources)
    .dst(destinations)
    .execute()
    .display()
)


In [5]:
# The beauty of fluent api: you can "just use the language"
scope_and_run = lambda x: \
  x.from_files_matching("db/version_set*").select().execute()

(project
    .flows()
    # Flows from if statement conditions
    .src(scope_and_run(
        project
        .query()
        .ifs()
        .conditions()
    ))
    # To variables (that have integral type)
    .dst(scope_and_run(
        project
        .query()
        .variables()
        .integers()
        .make_visualizable()
    ))
    .execute()
    .display()
)

In [2]:
namespaces = (project
    .query()
    .namespaces()
    .name_contains('config')
    .make_visualizable()
    .declarations()
    .select_usages()
    .execute()
    .display()
)

In [4]:
namespaces = (project
    .query()
    .namespaces()
    .name_semantically_similar_to('settings')
    .make_visualizable()
    .declarations()
    .select()
    .execute()
    .display()
)

In [4]:
project.deallocate(keep_db=False)